In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
import random
import time
import yaml
from klqr import *
from train_utils import *

In [2]:
with open('config.yml','r') as ymlfile:
    config = yaml.load(ymlfile)
    
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [3]:
env = gym.make('Pendulum-v0')
agent = klqr(config,sess)
agent.build_model()
train_agent(agent,env,max_timesteps=100000)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
z shape: (?, 6)
********** Iteration 1 ************
	Last Episode Reward: 0
********** Iteration 2 ************
	Last Episode Reward: -1125
********** Iteration 3 ************
	Last Episode Reward: -1125
********** Iteration 4 ************
	Last Episode Reward: -1187
********** Iteration 5 ************
	Last Episode Reward: -1187
********** Iteration 6 ************
	Last Episode Reward: -1181
********** Iteration 7 ************
	Last Episode Reward: -1181
********** Iteration 8 ************
	Last Episode Reward: -1106
********** Iteration 9 ************
	Last Episode Reward: -1106
********** Iteration 10 ************
	Last Episode Reward: -1143
********** Iteration 11 ************
	Last Episode Reward: -1143
********** Iteration 12 ************
	Last Episode Reward: -909
********** Iteration

KeyboardInterrupt: 